In [18]:
import pywt
import os
from pylab import *
from numpy import*
import pandas as pd

from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv('train/train_meta.csv')

In [3]:
df.head()

,ecg_id,patient_id,age,sex,height,weight,nurse,site,device,recording_date,...,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr,record_name,group
0,9,18792.0,55.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-12-08 09:44:43,...,", I-AVR,",NaN,NaN,NaN,NaN,10,records100/00000/00009_lr,records500/00000/00009_hr,00009_hr,1
1,34,13619.0,56.0,0,NaN,NaN,2.0,0.0,CS-12 E,1985-02-10 12:04:03,...,", alles,",NaN,NaN,NaN,NaN,9,records100/00000/00034_lr,records500/00000/00034_hr,00034_hr,3
2,43,11315.0,25.0,1,NaN,63.0,2.0,0.0,CS-12 E,1985-03-14 09:38:53,...,NaN,NaN,NaN,NaN,NaN,8,records100/00000/00043_lr,records500/00000/00043_hr,00043_hr,3
3,52,18153.0,35.0,0,NaN,82.0,2.0,0.0,CS-12 E,1985-05-18 08:15:54,...,NaN,NaN,NaN,NaN,NaN,1,records100/00000/00052_lr,records500/00000/00052_hr,00052_hr,2
4,57,16063.0,26.0,0,NaN,93.0,2.0,0.0,CS-12 E,1985-06-06 11:32:43,...,NaN,NaN,NaN,NaN,NaN,10,records100/00000/00057_lr,records500/00000/00057_hr,00057_hr,3


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2101 entries, 0 to 2100
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ecg_id                        2101 non-null   int64  
 1   patient_id                    2101 non-null   float64
 2   age                           2101 non-null   float64
 3   sex                           2101 non-null   int64  
 4   height                        689 non-null    float64
 5   weight                        905 non-null    float64
 6   nurse                         1947 non-null   float64
 7   site                          2100 non-null   float64
 8   device                        2101 non-null   object 
 9   recording_date                2101 non-null   object 
 10  report                        2101 non-null   object 
 11  scp_codes                     2101 non-null   object 
 12  heart_axis                    1231 non-null   object 
 13  inf

In [5]:
gts = pd.read_csv('train/train_gts.csv')

In [6]:
myocard_list = list(gts['myocard'])
df.insert(loc=len(df.columns), column='myocard', value=myocard_list)

In [7]:
d1 = list(df['static_noise'])
d2 = list(df['burst_noise'])
d3 = list(df['electrodes_problems'])
d4 = list(df['baseline_drift'])

In [8]:
df = df[(df.static_noise != d1) & (df.burst_noise != d2) & (df.electrodes_problems != d3) & (df.baseline_drift != d4)]
#удаление строк, содержащих шум, проблемы с электродами, смещение базовой линии/скачок

In [11]:
df[['weight', 'height']] = df[['weight', 'height']].fillna(df[['weight', 'height']].mean())

combo_info = df['weight'].astype(str) + '_' + df['height'].astype(str) + '_' + df['age'].astype(str) + '_' + df['sex'].astype(str)
df.insert(0, 'combo_info', combo_info)
df['combo_info'] = df.groupby('combo_info')['myocard'].transform('mean')
#объединение данных веса, роста, возраста и пола(во избежание переобучения)

In [12]:
df['scp_codes'] = df.groupby('scp_codes')['myocard'].transform('mean') #стандарт для обмена цифровыми ЭКГ

df['infarction_stadium1'] = df.groupby('infarction_stadium1')['myocard'].transform('mean') #стадия инфаркта
df['infarction_stadium2'] = df.groupby('infarction_stadium2')['myocard'].transform('mean') #вторая стадия инфаркта
df[['infarction_stadium1', 'infarction_stadium2']] = df[['infarction_stadium1', 'infarction_stadium2']].fillna(0)
#заполнение пустых значений нулём

df['heart_axis'] = df.groupby('heart_axis')['myocard'].transform('mean') #электрическая ось сердца
oft_value1 = df['heart_axis'].value_counts().idxmax() 
df['heart_axis'] = df['heart_axis'].fillna(oft_value1) #заполнение NaN наиболее встречающимся значением

df['device'] = df.groupby('device')['myocard'].transform('mean') #устройство записи
df['report'] = df.groupby('report')['myocard'].transform('mean') #краткий доклад о экг пациента

df['extra_beats'] = df.groupby('extra_beats')['myocard'].transform('mean')
#дополнительные удары сердца(обычно никак не влияют на здоров/болен)
df['extra_beats'] = df['extra_beats'].fillna(0)

oft_value2 = df['nurse'].value_counts().idxmax() #"номер" медсестры
df['nurse'] = df['nurse'].fillna(oft_value2)

oft_value3 = df['validated_by'].value_counts().idxmax() #"номер" утверждающего кардиолога
df['validated_by'] = df['validated_by'].fillna(oft_value3)

df['pacemaker'] = df.groupby('pacemaker')['myocard'].transform('mean')
#кардиостимулятор(регулирует функцию электрической проводящей системы сердца)
df['pacemaker'] = df['pacemaker'].fillna(0)

oft_value4 = df['site'].value_counts().idxmax() #"номер" сайта записи
df['site'] = df['site'].fillna(oft_value4)

In [13]:
df.drop(['recording_date', 'filename_lr', 'filename_hr', 'record_name',
         'age', 'sex', 'height', 'weight', 'static_noise', 
         'burst_noise', 'electrodes_problems', 'baseline_drift'], axis= 1 , inplace= True )

In [14]:
df.head()

,combo_info,ecg_id,patient_id,nurse,site,device,report,scp_codes,heart_axis,infarction_stadium1,infarction_stadium2,validated_by,second_opinion,initial_autogenerated_report,validated_by_human,extra_beats,pacemaker,strat_fold,group,myocard
2,0.0,43,11315.0,2.0,0.0,0.229167,0.3,0.219355,0.193798,0.0,0.0,0.0,False,False,True,0.0,0.0,8,3,0
3,0.0,52,18153.0,2.0,0.0,0.229167,0.0,0.190476,0.193798,0.0,0.0,0.0,False,False,True,0.0,0.0,1,2,0
4,0.0,57,16063.0,2.0,0.0,0.229167,0.3,0.219355,0.193798,0.0,0.0,0.0,False,False,True,0.0,0.0,10,3,0
5,0.0,61,14660.0,2.0,0.0,0.229167,0.3,0.219355,0.193798,0.0,0.0,0.0,False,False,True,0.0,0.0,6,1,0
8,0.0,108,11810.0,0.0,0.0,0.229167,0.0,0.352941,0.193798,0.0,0.0,0.0,False,False,True,0.0,0.0,8,3,0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1633 entries, 2 to 2100
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   combo_info                    1633 non-null   float64
 1   ecg_id                        1633 non-null   int64  
 2   patient_id                    1633 non-null   float64
 3   nurse                         1633 non-null   float64
 4   site                          1633 non-null   float64
 5   device                        1633 non-null   float64
 6   report                        1633 non-null   float64
 7   scp_codes                     1633 non-null   float64
 8   heart_axis                    1633 non-null   float64
 9   infarction_stadium1           1633 non-null   float64
 10  infarction_stadium2           1633 non-null   float64
 11  validated_by                  1633 non-null   float64
 12  second_opinion                1633 non-null   bool   
 13  initial_

In [25]:
df.corr()['myocard']

combo_info                      0.748284
ecg_id                          0.000240
patient_id                     -0.003604
nurse                           0.001220
site                           -0.011411
device                          0.050600
report                          0.779406
scp_codes                       0.616496
heart_axis                      0.053469
infarction_stadium1            -0.013247
infarction_stadium2                  NaN
validated_by                   -0.013426
second_opinion                  0.026957
initial_autogenerated_report   -0.021293
validated_by_human              0.014324
extra_beats                     0.099341
pacemaker                      -0.014135
strat_fold                      0.025879
group                           0.028105
myocard                         1.000000
Name: myocard, dtype: float64

In [16]:
values = df.values
X = values[:,0:18]
y = values[:,19]
y=y.astype('int')

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.35, random_state=0)

In [21]:
cat = CatBoostClassifier(depth=8,  learning_rate=0.05)
cat.fit(X_train, y_train)

0:	learn: 0.6167517	total: 127ms	remaining: 2m 6s
1:	learn: 0.5428700	total: 136ms	remaining: 1m 7s
2:	learn: 0.4880502	total: 144ms	remaining: 48s
3:	learn: 0.4381187	total: 152ms	remaining: 37.9s
4:	learn: 0.4026607	total: 161ms	remaining: 32s
5:	learn: 0.3563733	total: 169ms	remaining: 27.9s
6:	learn: 0.3244671	total: 176ms	remaining: 25s
7:	learn: 0.2905341	total: 183ms	remaining: 22.7s
8:	learn: 0.2641386	total: 191ms	remaining: 21s
9:	learn: 0.2465584	total: 198ms	remaining: 19.6s
10:	learn: 0.2277389	total: 205ms	remaining: 18.4s
11:	learn: 0.2094534	total: 212ms	remaining: 17.5s
12:	learn: 0.1961224	total: 220ms	remaining: 16.7s
13:	learn: 0.1849655	total: 227ms	remaining: 16s
14:	learn: 0.1739125	total: 234ms	remaining: 15.4s
15:	learn: 0.1644354	total: 241ms	remaining: 14.8s
16:	learn: 0.1592868	total: 248ms	remaining: 14.3s
17:	learn: 0.1528771	total: 255ms	remaining: 13.9s
18:	learn: 0.1444440	total: 262ms	remaining: 13.5s
19:	learn: 0.1374272	total: 269ms	remaining: 13.2s


174:	learn: 0.0144534	total: 1.35s	remaining: 6.36s
175:	learn: 0.0143152	total: 1.36s	remaining: 6.35s
176:	learn: 0.0142248	total: 1.36s	remaining: 6.34s
177:	learn: 0.0140801	total: 1.37s	remaining: 6.33s
178:	learn: 0.0139558	total: 1.38s	remaining: 6.32s
179:	learn: 0.0138752	total: 1.39s	remaining: 6.31s
180:	learn: 0.0137603	total: 1.39s	remaining: 6.3s
181:	learn: 0.0136256	total: 1.4s	remaining: 6.29s
182:	learn: 0.0135494	total: 1.41s	remaining: 6.28s
183:	learn: 0.0134651	total: 1.41s	remaining: 6.27s
184:	learn: 0.0133838	total: 1.42s	remaining: 6.25s
185:	learn: 0.0132967	total: 1.43s	remaining: 6.25s
186:	learn: 0.0131783	total: 1.43s	remaining: 6.24s
187:	learn: 0.0130715	total: 1.44s	remaining: 6.23s
188:	learn: 0.0129750	total: 1.45s	remaining: 6.21s
189:	learn: 0.0128905	total: 1.46s	remaining: 6.2s
190:	learn: 0.0127603	total: 1.46s	remaining: 6.19s
191:	learn: 0.0126461	total: 1.47s	remaining: 6.18s
192:	learn: 0.0125457	total: 1.48s	remaining: 6.17s
193:	learn: 0.0

341:	learn: 0.0048547	total: 2.8s	remaining: 5.39s
342:	learn: 0.0048390	total: 2.81s	remaining: 5.39s
343:	learn: 0.0048006	total: 2.82s	remaining: 5.39s
344:	learn: 0.0047733	total: 2.83s	remaining: 5.38s
345:	learn: 0.0047539	total: 2.84s	remaining: 5.37s
346:	learn: 0.0047290	total: 2.85s	remaining: 5.37s
347:	learn: 0.0047080	total: 2.86s	remaining: 5.36s
348:	learn: 0.0046881	total: 2.87s	remaining: 5.35s
349:	learn: 0.0046734	total: 2.88s	remaining: 5.35s
350:	learn: 0.0046516	total: 2.89s	remaining: 5.34s
351:	learn: 0.0046311	total: 2.9s	remaining: 5.33s
352:	learn: 0.0046120	total: 2.91s	remaining: 5.33s
353:	learn: 0.0045954	total: 2.92s	remaining: 5.32s
354:	learn: 0.0045684	total: 2.92s	remaining: 5.32s
355:	learn: 0.0045465	total: 2.93s	remaining: 5.31s
356:	learn: 0.0045216	total: 2.94s	remaining: 5.3s
357:	learn: 0.0045015	total: 2.95s	remaining: 5.29s
358:	learn: 0.0044880	total: 2.96s	remaining: 5.29s
359:	learn: 0.0044734	total: 2.97s	remaining: 5.28s
360:	learn: 0.0

502:	learn: 0.0028716	total: 4.23s	remaining: 4.18s
503:	learn: 0.0028637	total: 4.24s	remaining: 4.17s
504:	learn: 0.0028563	total: 4.25s	remaining: 4.16s
505:	learn: 0.0028481	total: 4.26s	remaining: 4.16s
506:	learn: 0.0028400	total: 4.27s	remaining: 4.15s
507:	learn: 0.0028289	total: 4.28s	remaining: 4.15s
508:	learn: 0.0028177	total: 4.29s	remaining: 4.14s
509:	learn: 0.0028070	total: 4.3s	remaining: 4.13s
510:	learn: 0.0028003	total: 4.3s	remaining: 4.12s
511:	learn: 0.0027909	total: 4.32s	remaining: 4.11s
512:	learn: 0.0027837	total: 4.32s	remaining: 4.1s
513:	learn: 0.0027773	total: 4.33s	remaining: 4.1s
514:	learn: 0.0027706	total: 4.34s	remaining: 4.09s
515:	learn: 0.0027603	total: 4.35s	remaining: 4.08s
516:	learn: 0.0027541	total: 4.36s	remaining: 4.07s
517:	learn: 0.0027451	total: 4.37s	remaining: 4.06s
518:	learn: 0.0027350	total: 4.38s	remaining: 4.06s
519:	learn: 0.0027249	total: 4.38s	remaining: 4.05s
520:	learn: 0.0027206	total: 4.39s	remaining: 4.04s
521:	learn: 0.00

681:	learn: 0.0019820	total: 5.87s	remaining: 2.74s
682:	learn: 0.0019786	total: 5.88s	remaining: 2.73s
683:	learn: 0.0019752	total: 5.89s	remaining: 2.72s
684:	learn: 0.0019704	total: 5.9s	remaining: 2.71s
685:	learn: 0.0019663	total: 5.91s	remaining: 2.7s
686:	learn: 0.0019620	total: 5.92s	remaining: 2.69s
687:	learn: 0.0019576	total: 5.93s	remaining: 2.69s
688:	learn: 0.0019543	total: 5.93s	remaining: 2.68s
689:	learn: 0.0019518	total: 5.94s	remaining: 2.67s
690:	learn: 0.0019474	total: 5.95s	remaining: 2.66s
691:	learn: 0.0019431	total: 5.96s	remaining: 2.65s
692:	learn: 0.0019390	total: 5.97s	remaining: 2.64s
693:	learn: 0.0019363	total: 5.98s	remaining: 2.63s
694:	learn: 0.0019318	total: 5.99s	remaining: 2.63s
695:	learn: 0.0019285	total: 5.99s	remaining: 2.62s
696:	learn: 0.0019245	total: 6s	remaining: 2.61s
697:	learn: 0.0019205	total: 6.01s	remaining: 2.6s
698:	learn: 0.0019178	total: 6.02s	remaining: 2.59s
699:	learn: 0.0019136	total: 6.03s	remaining: 2.58s
700:	learn: 0.0019

843:	learn: 0.0015363	total: 7.3s	remaining: 1.35s
844:	learn: 0.0015345	total: 7.31s	remaining: 1.34s
845:	learn: 0.0015321	total: 7.32s	remaining: 1.33s
846:	learn: 0.0015301	total: 7.33s	remaining: 1.32s
847:	learn: 0.0015276	total: 7.34s	remaining: 1.31s
848:	learn: 0.0015253	total: 7.35s	remaining: 1.31s
849:	learn: 0.0015238	total: 7.36s	remaining: 1.3s
850:	learn: 0.0015222	total: 7.37s	remaining: 1.29s
851:	learn: 0.0015203	total: 7.37s	remaining: 1.28s
852:	learn: 0.0015177	total: 7.38s	remaining: 1.27s
853:	learn: 0.0015159	total: 7.39s	remaining: 1.26s
854:	learn: 0.0015141	total: 7.4s	remaining: 1.25s
855:	learn: 0.0015124	total: 7.41s	remaining: 1.25s
856:	learn: 0.0015102	total: 7.42s	remaining: 1.24s
857:	learn: 0.0015085	total: 7.42s	remaining: 1.23s
858:	learn: 0.0015069	total: 7.43s	remaining: 1.22s
859:	learn: 0.0015035	total: 7.44s	remaining: 1.21s
860:	learn: 0.0015015	total: 7.45s	remaining: 1.2s
861:	learn: 0.0014997	total: 7.47s	remaining: 1.2s
862:	learn: 0.001

In [22]:
f1_score(y_train, cat.predict(X_train), average = 'macro')

1.0

In [23]:
f1_score(y_test, cat.predict(X_test), average = 'macro')

0.9395509007598828